<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/bmulas1535/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/bmulas1535/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be sometime that can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

## Importing the data

```
!unzip data.zip
```

In [0]:
#!unzip data.zip

In [0]:
import os
from PIL import Image
import numpy as np

In [0]:
forest_images = list()
# Collect forest data
for picture in os.listdir('data/forest'):
  if 'Thumbs' in picture:
    pass
  if picture == '.ipynb_checkpoints':
    pass
  else:
    image = Image.open(f'data/forest/{picture}')
    forest_images.append(image)


In [0]:
mountain_images = list()
# Collect mountain data
for picture in os.listdir('data/mountain'):
  if 'Thumbs' in picture:
    pass
  if picture == '.ipynb_checkpoints':
    pass
  else:
    image = Image.open(f'data/mountain/{picture}')
    mountain_images.append(image)


In [0]:
# Create target label for images
y_forest = [0 for x in range(len(forest_images))]
y_mountain = [1 for x in range(len(mountain_images))]


In [0]:
# Get arrays into list
forest_arrays = list()
mountain_arrays = list()

for image in forest_images:
  forest_arrays.append(np.asarray(image))

for image in mountain_images:
  mountain_arrays.append(np.asarray(image))


In [0]:
# Concatentate label lists
train_labels = y_forest + y_mountain


In [0]:
# Concatenate image array lists
train_images = forest_arrays + mountain_arrays


In [0]:
train_images = np.array(train_images)

In [0]:
train_images = train_images/255

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, \
decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

## Instatiate Model

In [12]:
# Instantiate resnet model
resnet = ResNet50(weights='imagenet', include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Freeze resnet layers
for layer in resnet.layers:
    layer.trainable = False

In [0]:

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [0]:
# compile the concatenated model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [21]:
# Fit the model
model.fit(train_images, train_labels, epochs=200)

Train on 702 samples
Epoch 1/200
702/702 [==============================] - 3s 4ms/sample - loss: 0.2416 - acc: 0.9345
Epoch 2/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.1521 - acc: 0.9672
Epoch 3/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.0646 - acc: 0.9758
Epoch 4/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.0155 - acc: 0.9929
Epoch 5/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.2251 - acc: 0.9630
Epoch 6/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.1415 - acc: 0.9615
Epoch 7/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.0365 - acc: 0.9872
Epoch 8/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.0156 - acc: 0.9986
Epoch 9/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.0573 - acc: 0.9886
Epoch 10/200
702/702 [==============================] - 2s 3ms/sample - loss: 0.1457 - acc: 0.974

In [0]:
train_images[0].shape

## Fit Model

# Custom CNN Model

In [0]:
# Compile Model


In [0]:
# Fit Model


# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language